<a href="https://colab.research.google.com/github/pythonkvs/seminars/blob/main/%D0%9C%D0%BE%D0%B4%D1%83%D0%BB%D0%B8_%D0%BF%D0%B0%D0%BA%D0%B5%D1%82%D1%8B_30_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://compscicenter.ru/courses/python/2015-autumn/classes/1556/

# Модули

* `useful.py`
```python
"""I'm a useful module."""
some_variable = "foobar"
def boo():
    return 42
```

In [ ]:
with open('useful.py', 'w') as fout:
    fout.write('\"\"\"I\'m a useful module.\"\"\"' + '\n')
    fout.write('some_variable = \"foobar\"' + '\n')
    fout.write('def boo():' + '\n')
    fout.write('\treturn 42' + '\n')

## Файл == модуль

* Модулем называется файл с расширением py.  
* Каждый модуль задаёт новое пространство имён, атрибуты
которого соответствуют именам, определённым в файле:




In [ ]:
import useful
dir(useful)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'boo',
 'some_variable']

Кроме явно определённых имён в модуле содержатся:

In [ ]:
useful.__name__

'useful'

In [ ]:
useful.__doc__

"I'm a useful module."

In [ ]:
useful.__file__

'/content/useful.py'

In [ ]:
useful.__cached__ # и другие

'/content/__pycache__/useful.cpython-37.pyc'

## `__name__ == "__main__"`

* Модуль можно выполнить, передав его в качестве
аргумента интерпретатору.  
* В этом случае переменная `__name__` внутри модуля будет
иметь специальное значение `"__main__"`.  
* Пример:
```python
# useful.py
def test():
    assert boo() == 4
if __name__ == "__main__":
    print("Running tests ... ")
    test()
    print("OK")
```
```bash
$ python ./useful.py
Running tests ...
OK
```

## Оператор `import`

Оператор `import` “импортирует” модуль с указанным
именем и создаёт на него ссылку в текущей области
видимости:

In [ ]:
import useful # исполняет модуль сверху вниз
useful

<module 'useful' from '/content/useful.py'>

С помощью оператора `as` можно изменить имя
переменной, в которую будет записана ссылка на модуль:

In [ ]:
import useful as alias
alias

<module 'useful' from '/content/useful.py'>

Чтобы модуль был доступен для импорта, содержащая его
директория должна присутствовать в списке `sys.path`:

In [ ]:
import sys
sys.path

['',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython']

## Оператор `from...import`

Оператор `from ... import` импортирует имя из другого
модуля в текущую область видимости:

In [ ]:
from useful import boo
boo()

42

Синтаксис оператора позволяет перечислить несколько
имен через запятую и, возможно, переименовать
некоторые из них:

In [ ]:
from useful import boo as foo, some_variable
foo()

42

In [ ]:
some_variable

'foobar'

## “Семантика” оператора `from...import`

Оператор `from ... import` можно однозначно переписать
через оператор `import`:

In [ ]:
from useful import boo as foo, some_variable
# HARDCORE REWRITING MAGIC
import useful
foo = useful.boo
some_variable = useful.some_variable
del useful

Всё сказанное про оператор `import` релевантно и для
оператора `from ... import`.

## Оператор `from...import *`

В качестве второго аргумента оператора `from ... import`
можно указать *.  

Если в модуле определена глобальная переменная
`__all__`, то будут импортированы только те имена, которые
в ней перечислены.  

Иначе — все имена из `globals()` модуля.

In [ ]:
from useful import *
some_variable

'foobar'

На практике оператор `from ... import *` используют
редко, потому что он затрудняет чтение кода.

## Модули: резюме

*   Модуль в Python — это просто файл с расширением py.
*   Модуль можно импортировать целиком или выборочно с
помощью операторов `import` и `from ... import`.
*   В момент импорта байт-код модуля выполняется
интерпретатором сверху вниз.
*   Три правила импортирования модулей:
    *   размещайте все импорты в начале модуля,
    *   сортируйте их в лексикографическом порядке,
    *   располагайте блок `import` перед `from ... import`.
*   Пример:


In [ ]:
import os
import sys
from collections import OrderedDict
from itertools import islice

# Пакеты

## Пакет == директория с модулями

* Пакеты позволяют структурировать код на Python.  

* Любая директория, содержащая файл `__init__.py`,
автоматически становится пакетом.  

* В качестве примера рассмотрим  
`useful`  
├── `__init__.py` # !  
├── `bar.py`  
└── `foo.py`  

* Импортируем пакет `useful`:
```python
>>> import useful
>>> useful
<module 'useful' from './useful/__init__.py'>
>>> useful.foo
Traceback (most recent call last):
File "<stdin>", line 1, in <module>
AttributeError: 'module' object has no attribute 'foo'
```

## Импорт модулей из пакета

* При импорте пакета импортируется только `__init__.py`.
```python
>>> import useful.foo
>>> useful # !
<module 'useful' from './useful/__init__.py'>
>>> useful.foo
<module 'useful.foo' from './useful/foo.py'>
```

* Остальные модули необходимо импортировать явно:
```python
>>> useful.bar
Traceback (most recent call last):
File "<stdin>", line 1, in <module>
AttributeError: 'module' object has no attribute 'bar'
>>> from useful import bar
>>> bar
<module 'useful.bar' from './useful/bar.py'>
```

## Относительный импорт

* Примеры, которые мы видели ранее, использовали
абсолютный импорт — вызов оператора `import` содержал
имя пакета:
```python
import useful.foo
from useful import bar
```

* Можно (и нужно!) использовать относительный импорт:
```python
from . import foo, bar
# ^ соответствует имени пакета, в котором
# вызывается импорт
```

Почему? Не нужно изменять импорты при
переименовании или перемещении пакета.

* Одно но: не работает в интерактивной оболочке:
```python
>>> from . import useful
Traceback (most recent call last):
File "<stdin>", line 1, in <module>
SystemError: Parent module '' not loaded, [...]
```

## Вложенные пакеты aka sub-packages

* Внутри пакетов могут находиться не только модули, но и
другие пакеты. Сделаем модуль `bar` пакетом:  
<pre>
useful
├── __init__.py
├── bar
│   ├── __init__.py
│   ├── boo.py 
└── foo.py
</pre>



* Синтаксически работа с вложенным пакетом `useful.bar`
ничем не отличается от работы с его предшественником:
```python
>>> import useful.bar
>>> useful.bar
<module 'useful.bar' from './useful/bar/__init__.py'>
```

* Замечание: в модуле `useful.bar.boo` тоже можно
использовать относительный импорт:
```python
from . import something
from ..foo import something_else
```

## Ещё немного об `__init__.py`: фасад

*   Задача модуля `__init__.py` — инициализировать пакет,
поэтому не стоит реализовывать в нём всю логику.
*   Что стоит делать в `__init__.py`?
    *   Ничего.
    *   Объявить глобальные для пакета переменные (может быть).
    *   Оградить пакет фасадом, то есть импортировать имена из
вложенных модулей и пакетов и определить `__all__`.


*   Фасад для пакета `useful`:
<pre>
useful
├── __init__.py
├── bar
│   ├── __init__.py
│   ├── boo.py
└── foo.py
</pre>



* `useful/bar/__init__.py`
```python
# useful/bar/__init__.py
from .boo import *
__all__ = boo.__all__
# useful/__init__.py
from .foo import *
from .bar import *
__all__ = foo.__all__ + bar.__all__
```

## Плюсы и минусы использования фасада в `__init__.py`

*   Плюсы:
    *   Пользователю не нужно запоминать внутреннюю структуру
пакета и думать, с чем он работает: модулем или пакетом.
```python
        *   from urllib import urlopen или
        *   from urllib.request import urlopen или
        *   from urllib.requests import urlopen?
```
    *   Интерфейс не зависит от деталей реализации — можно
перемещать код между внутренними модулями и пакетами.

*   Минусы?


In [ ]:
%time import sympy
%time import theano

CPU times: user 386 ms, sys: 70.9 ms, total: 457 ms
Wall time: 826 ms
CPU times: user 554 ms, sys: 170 ms, total: 724 ms
Wall time: 2.51 s


## Исполняемые модули и пакеты

* Любой модуль можно выполнить как скрипт, передав его
имя в качестве аргумента `-m`:
```python
# useful/foo.py
print(__name__)
```
```bash
$ python -m useful.foo
'__main__' # !
```
* Чтобы пакет был исполняемым, в нём должен быть файл
`__main__.py`:
```python
# useful/__main__.py
print("It works!")
```
```bash
$ python -m useful
useful.__init__ # ?
It works!
```
```python
# useful/__init__.py
print("useful.__init__")
```
```bash
$ python -m useful.__main__
useful.__init__
It works!
```

## Пакеты: резюме

*   Пакеты — это способ группировать код на Python.
*   Любая директория, содержащая файл `__init__.py`, задаёт
пакет.
*   Полезные детали, о которых стоит помнить:
    *   в пакете можно (и нужно!) использовать относительный
импорт вместо абсолютного;
    *   с помощью `__init__.py` можно абстрагировать детали
реализации пакета от пользователя,
    *   а добавив файл `__main__.py` — сделать пакет
исполняемым.